In [25]:
import os

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/anshu1016/DataScienceProject-BiolerPlate.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'anshu1016'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '9bbc4bf2245f9d25bba3b6f9653f4609f8ce312a'

In [26]:
%pwd

'c:\\ML_OPS\\Projects\\project1'

In [27]:
# os.chdir('../')
%pwd

'c:\\ML_OPS\\Projects\\project1'

In [28]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mflow_uri: str

In [29]:
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.shared import read_yaml, create_directories,save_json


In [30]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet 
        schema = self.schema.TARGET_COLUMN


        create_directories([config.root_dir])
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            all_params = params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mflow_uri = os.getenv('MLFLOW_TRACKING_URI')
        )
        return model_evaluation_config

In [31]:
import pandas as pd  
import numpy as np  
import os
import mlflow
import mlflow.sklearn
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from urllib.parse import urlparse

In [32]:
class ModelEvaluation:
    def __init__(self,config:ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self,actual,pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop(columns=[self.config.target_column])
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mflow_uri)
        tracking_url_type_store = urlparse(self.config.mflow_uri).scheme


        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse,mae,r2) = self.eval_metrics(test_y, predicted_qualities)

            # sAVING mETRICS AS lOCAL
            scores = {'rmse':rmse,'mae':mae, 'r2':r2}

            save_json(path=Path(self.config.metric_file_name),data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('rmse: ',rmse)
            mlflow.log_metric('mae: ',mae)
            mlflow.log_metric('r2: ',r2)
            

            # Model Registry does not work with file store
            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model,artifact_path="model",registered_model_name='Elasticnet Model')
            else:
                mlflow.sklearn.log_model(model,'model')



In [33]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e 


[2025-06-06 14:22:52,393 : INFO : shared : Yaml File: config\config.yaml loaded successfully!]
[2025-06-06 14:22:52,393 : INFO : shared : Yaml File: params.yaml loaded successfully!]
[2025-06-06 14:22:52,399 : INFO : shared : Yaml File: schema.yaml loaded successfully!]
[2025-06-06 14:22:52,402 : INFO : shared : created directory at : artifacts]
[2025-06-06 14:22:52,403 : INFO : shared : created directory at : artifacts/model_evaluation]
[2025-06-06 14:22:54,740 : INFO : shared : json file saved at: artifacts\model_evaluation\metrics.json]
🏃 View run amazing-loon-5 at: https://dagshub.com/anshu1016/DataScienceProject-BiolerPlate.mlflow/#/experiments/0/runs/ce0e513ae47d4d71a4f587210d788869
🧪 View experiment at: https://dagshub.com/anshu1016/DataScienceProject-BiolerPlate.mlflow/#/experiments/0


TypeError: log_model() missing 1 required positional argument: 'artifact_path'